In [1]:
# 
import pandas as pd
import re

# Convert NDJSON

In [12]:
productionData = pd.DataFrame()
paragraphs = []
titles = []
ids = []
isLessons = []
sources = []
annotationTitles = []
annotationStatuses = []
annotationSummaries = []
lastAnnotateds = []
annotatedBys = []
f = open("../data/sentences production 20200320.json", "r")

startTextParagraph = '"paragraph":"'
endTextParagraph = '","themes"'

startTextId = '"_id":"'
endTextId = '","_score"'

# startTextThemes = '"themes":'
# endTextThemes = ',"sectors":'

# startTextSectors = ',"sectors":'
# endTextSectors = ',"countries":'

# startTextCountries = ',"countries":'
# endTextCountries = ',"lessonStrength":'


startTextIsLesson = '"isLesson":'
endTextIsLesson = ',"month":"'


# c = 0
for line in f.readlines():
#     c = c + 1
#     print(c)

    startTextATitle = '"annotationTitle":"'
    endTextAtitle = '","summary":'

    startTextASummary = '"annotationSummary":"'
    endTextASummary = '","lastAnnotated":'

    startTextAStatus = '"annotationStatus":"'
    endTextAStatus = '","urlToFile":'

    startTextALast = '"lastAnnotated":"'
    endTextALast = '","annotationStatus":'

    startTextABy = 'annotatedBy":"'
    endTextABy = '","context":'
    
    startTextSource = '"source":"'
    endTextSource = '"}}'
    
    startTextTitle = '"title":"'
    endTextTitle = '","paragraph":'
    
    paragraph = line[line.index(startTextParagraph) + 13: line.index(endTextParagraph)]
    title = line[line.index(startTextTitle) + len(startTextTitle): line.index(endTextTitle)]
    paragraphId = line[line.index(startTextId) + 7: line.index(endTextId)]
    isLesson = line[line.index(startTextIsLesson) + 11: line.index(endTextIsLesson)]
    annotationStatus = line[line.index(startTextAStatus) + len(startTextAStatus): line.index(endTextAStatus)]
    
    if(not startTextATitle in line):
        annotationTitle = ""
    else:
        annotationTitle = line[line.index(startTextATitle) + len(startTextATitle): line.index(endTextAtitle)]
        
    if(not startTextASummary in line):
        annotationSummary = ""
    else:
        annotationSummary = line[line.index(startTextASummary) + len(startTextASummary): line.index(endTextASummary)]
        
    if(not startTextALast in line):
        lastAnnotated = ""
    else:
        lastAnnotated = line[line.index(startTextALast) + len(startTextALast): line.index(endTextALast)]

    if(not startTextABy in line):
        annotatedBy = ""
    else:
        indexEndTextABy = line.index(endTextABy)
        indexStartTextABy = line.index(startTextABy)
        if(indexEndTextABy < indexStartTextABy):
            endTextABy = '","annotationSummary":'
            indexEndTextABy = line.index(endTextABy)           
        annotatedBy = line[indexStartTextABy + len(startTextABy): indexEndTextABy]
    
    if(not endTextSource in line):
        endTextSource = '","validatedBy"'
    source = line[line.index(startTextSource) + 10: line.index(endTextSource)]
    
    paragraphs.append(paragraph)
    titles.append(title)
    annotationTitles.append(annotationTitle)
    annotationSummaries.append(annotationSummary)
    lastAnnotateds.append(lastAnnotated)
    annotatedBys.append(annotatedBy)
    annotationStatuses.append(annotationStatus)
    ids.append(paragraphId)
    isLessons.append(isLesson)
    sources.append(source)
f.close()

productionData['paragraph'] = paragraphs
productionData['title'] = titles
productionData['id'] = ids
productionData['isLesson'] = isLessons
productionData['source'] = sources
productionData['annotationTitle'] = annotationTitles
productionData['annotationSummary'] = annotationSummaries
productionData['annotationStatus'] = annotationStatuses
productionData['lastAnnotated'] = lastAnnotateds
productionData['annotatedBy'] = annotatedBys
productionData.to_excel('../data/production_data_20200320.xlsx')

# Old Data

In [13]:
df = pd.read_excel("../data/merged_lessons_forecasted_data_reindex_deduped_20200320.xlsx")
# df = df.rename(columns={"Sentence": "paragraph"})
print(len(df))
print(df.columns)

55368
Index(['Unnamed: 0', 'Unnamed: 0.1', 'Countries', 'Download Link',
       'File Names', 'Lesson Type', 'Month', 'Project Number', 'Sectors',
       'Themes', 'Title', 'Year', 'annotatedBy', 'annotationStatus',
       'annotationSummary', 'annotationTitle', 'dataSource', 'id', 'isLesson',
       'lastAnnotated', 'paragraph', 'source', 'reference_id', 'Modalities',
       'machine generated', 'source_lesson', 'related_lessons',
       'Lesson Strength', 'topic', 'topTopics'],
      dtype='object')


# Production data

In [14]:
productionData = pd.read_excel("../data/production_data_20200320.xlsx")

In [15]:
productionData['annotatedBy'].unique()

array([nan, 'Ueda, Tomoo', 'Jovellanos, Jerome', 'Sandhu, Sonia Chand ',
       'Shimako Takahashi', 'Sonia Chand Sandhu ', 'Subramaniam Nathan',
       'Bloom. Erik', 'Bano, Alfredo', 'Perdana, Ari',
       'Sandhu, Sonia Chand', 'Hauck, Lauren', 'Mike Diza', 'Kim, Eungji',
       'Diza, Mike', 'Sonia Chand Sandhu', 'Morales Alvin', 'Bloom, Erik',
       'Bano, Leal', 'Ueda, Tonmoo', ' Sandhu, Sonia Chand',
       'Sandhu. Sonia Chand ', 'Erik Bloom', 'kolkma, walter',
       'Somma, Simona', 'Mike', 'Simona Somma', 'Takahashi, Shimako',
       'Brubaker, Andrew', 'Villena, Serge', 'Toshiyuki Yokota', 'sash',
       'Toshiyuki Yokota '], dtype=object)

# Get data with annotations

In [16]:
productionDataWithAnnotations = productionData[productionData['annotationStatus'] != 'draft']
print(len(productionDataWithAnnotations))
productionDataWithAnnotations = productionDataWithAnnotations[productionDataWithAnnotations['lastAnnotated'] != '']
print(len(productionDataWithAnnotations))
print(productionDataWithAnnotations['lastAnnotated'].unique())

359
359
['2/4/2020 9:15:38 AM' '2/25/2020 7:59:18 AM' '2/7/2020 7:54:08 AM'
 '2/25/2020 7:47:16 AM' '2/19/2020 12:25:32 AM' '2/19/2020 3:48:20 AM'
 '2/17/2020 12:54:03 PM' '2/11/2020 7:48:33 AM' '2/8/2020 11:53:41 AM'
 '2/8/2020 8:17:24 AM' '2/7/2020 8:44:33 AM' '2/7/2020 7:28:06 AM'
 '2/4/2020 9:15:19 AM' '2/3/2020 6:06:59 AM' '1/31/2020 6:12:56 AM'
 '1/31/2020 5:33:26 AM' '1/27/2020 9:34:48 AM' '1/31/2020 3:43:58 AM'
 '1/31/2020 3:24:55 AM' '1/30/2020 10:13:27 AM' '1/30/2020 1:16:42 AM'
 '1/27/2020 6:57:39 AM' '3/11/2020 5:53:23 AM' '3/18/2020 10:22:06 AM'
 '3/18/2020 10:25:32 AM' '3/18/2020 10:26:17 AM' '2/19/2020 12:44:25 AM'
 '2/26/2020 8:25:59 AM' '2/25/2020 6:34:46 AM' '2/26/2020 6:21:17 AM'
 '2/26/2020 6:11:09 AM' '2/26/2020 6:02:19 AM' '2/25/2020 3:20:51 AM'
 '2/26/2020 5:56:18 AM' '2/25/2020 3:18:39 AM' '2/26/2020 5:39:15 AM'
 '2/19/2020 3:51:20 AM' '2/19/2020 3:49:09 AM' '2/19/2020 3:48:45 AM'
 '2/23/2020 11:54:04 PM' '2/23/2020 11:52:51 PM' '2/19/2020 3:43:50 AM'
 '2/23/202

In [8]:
count = 0
for idOfUpdated in productionDataWithAnnotations['id'].tolist():
    if idOfUpdated in df['id'].tolist():
        count = count + 1
count

0

In [9]:
dfWithAnnotations = df[df['annotationStatus'] != 'draft']
len(dfWithAnnotations)

354

In [17]:
for index, row in productionDataWithAnnotations.iterrows():
    index = int(row['id'])
    df.loc[df['id'] == index, 'annotationStatus'] = row['annotationStatus']
    df.loc[df['id'] == index, 'annotatedBy'] = row['annotatedBy']
    df.loc[df['id'] == index, 'annotationTitle'] = row['annotationTitle']
    df.loc[df['id'] == index, 'annotationSummary'] = row['annotationSummary']
    df.loc[df['id'] == index, 'lastAnnotated'] = row['lastAnnotated']
    df.loc[df['id'] == index, 'isLesson'] = row['isLesson']

In [18]:
print(df['annotatedBy'].unique())

[nan 'Ueda, Tomoo' 'Jovellanos, Jerome' 'Sandhu, Sonia Chand '
 'Shimako Takahashi' 'Sonia Chand Sandhu ' 'Subramaniam Nathan'
 'Bloom. Erik' 'Bano, Alfredo' 'Perdana, Ari' 'Sandhu, Sonia Chand'
 'Hauck, Lauren' 'Mike Diza' 'Kim, Eungji' 'Diza, Mike'
 'Sonia Chand Sandhu' 'Morales Alvin' 'Bloom, Erik' 'Bano, Leal'
 'Ueda, Tonmoo' ' Sandhu, Sonia Chand' 'Sandhu. Sonia Chand ' 'Erik Bloom'
 'kolkma, walter' 'Somma, Simona' 'Mike' 'Simona Somma'
 'Takahashi, Shimako' 'Brubaker, Andrew' 'Villena, Serge'
 'Toshiyuki Yokota' 'sash' 'Toshiyuki Yokota ']


# Copy values of source

In [146]:
for index, row in productionData.iterrows():
    index = int(row['id'])
    df.loc[df['id'] == index, 'source'] = row['source']

In [152]:
df = df.replace('tagged","validatedBy":"Asquith, Joanne', 'tagged')

In [12]:
df.columns

Index(['Unnamed: 0', 'Countries', 'Download Link', 'File Names', 'Lesson Type',
       'Month', 'Project Number', 'Sectors', 'Themes', 'Title', 'Year',
       'annotatedBy', 'annotationStatus', 'annotationSummary',
       'annotationTitle', 'dataSource', 'id', 'isLesson', 'lastAnnotated',
       'paragraph', 'source', 'reference_id', 'Modalities',
       'machine generated', 'source_lesson', 'related_lessons',
       'Lesson Strength', 'topic', 'topTopics'],
      dtype='object')

# Update source if identified by model as lesson

In [163]:
# for index, row in df.iterrows():
#     if row['isLesson'] == True and row['machine generated'] == row['machine generated']:
#         df.loc[df['id'] == index, 'source'] = 'tagged'
#     else:
#         df.loc[df['id'] == index, 'source'] = 'PCR'

In [186]:
sources = []
summaries = df['machine generated'].tolist()
isLessons = df['isLesson'].tolist()
for summary, isLesson in zip(summaries, isLessons):
    if summary != summary:
        sources.append('tagged')
    else:
        sources.append('PCR')
df['source'] = sources

In [13]:
df['source'].value_counts()

tagged    44549
PCR       10819
Name: source, dtype: int64

In [19]:
df.to_excel('../data/merged_lessons_forecasted_data_reindex_deduped_20200323.xlsx')